In [22]:
import pandas as pd
import numpy as np

In [17]:
data = pd.read_csv('Final Project Baseline Values - Sheet1.csv', header = None)

In [18]:
data.columns = ['metrics', 'values']

In [20]:
data['metrics'] = ['unique cookies to view page/day', 'unique cookies to click "Start trial"/per day', 'enrollments/day',
                  'ctp on star trial', 'gross conversion', 'retention', 'net conversion']
data['dmin'] = [3000, 50, 240, 0.01, 0.01, 0.01, 0.0075]
data

,metrics,values,dmin
0,unique cookies to view page/day,40000.000000,3000.0000
1,"unique cookies to click ""Start trial""/per day",3200.000000,50.0000
2,enrollments/day,660.000000,240.0000
3,ctp on star trial,0.080000,0.0100
4,gross conversion,0.206250,0.0100
5,retention,0.530000,0.0100
6,net conversion,0.109313,0.0075


In [24]:
data['sample'] = np.nan
f = 5000 / 40000
for i in range(3):
    data.loc[i,'sample'] = data.loc[i, 'values'] * f
data

,metrics,values,dmin,sample
0,unique cookies to view page/day,40000.000000,3000.0000,5000.0
1,"unique cookies to click ""Start trial""/per day",3200.000000,50.0000,400.0
2,enrollments/day,660.000000,240.0000,82.5
3,ctp on star trial,0.080000,0.0100,NaN
4,gross conversion,0.206250,0.0100,NaN
5,retention,0.530000,0.0100,NaN
6,net conversion,0.109313,0.0075,NaN


In [29]:
def normality(p, n, metric):
    if p * n > 10 and n * (1-p) > 10:
        print(metric + ': normal distribution assumption is valid')
    else:
        print(metric + ': normal distribution assumption is not valid')
for i, j in zip([1,2,1], [4,5,6]):
    normality(data.loc[j, 'values'],data.loc[i, 'sample'], data.loc[j, 'metrics'] )

gross conversion: normal distribution assumption is valid
retention: normal distribution assumption is valid
net conversion: normal distribution assumption is valid


In [35]:
def std(p,n, metric):
    sd = round((p * (1-p)/n) ** 0.5 , 4)
    print(metric + ' standard deviation:',sd )
    return sd
    
data['sd'] = np.nan
for i,j in zip([1,2,1], [4,5,6]):
    data.loc[j, 'sd'] = std(data.loc[j, 'values'], data.loc[i, 'sample'], data.loc[j, 'metrics'])
data

gross conversion standard deviation: 0.0202
retention standard deviation: 0.0549
net conversion standard deviation: 0.0156


,metrics,values,dmin,sample,sd
0,unique cookies to view page/day,40000.000000,3000.0000,5000.0,NaN
1,"unique cookies to click ""Start trial""/per day",3200.000000,50.0000,400.0,NaN
2,enrollments/day,660.000000,240.0000,82.5,NaN
3,ctp on star trial,0.080000,0.0100,NaN,NaN
4,gross conversion,0.206250,0.0100,NaN,0.0202
5,retention,0.530000,0.0100,NaN,0.0549
6,net conversion,0.109313,0.0075,NaN,0.0156
